In [46]:
import os
from python_libs.landmarks import LandmarkSBR
from python_libs.video import get_video_info_file
from python_libs.video import get_frames
from python_libs.dataset import ReplayAttackParser
import pandas as pd
import time
import h5py
from tqdm import tqdm
import datetime

In [11]:
model_path = os.path.join(os.getcwd(), "..", "models", "cpm_vgg16-epoch-049-050.pth")
config_path = os.path.join(os.getcwd(), "..", "configs", "SBRDetector.config")
model = LandmarkSBR(model_path=model_path,config_path=config_path, cpu=False)

The general dataset initialization done : GeneralDataset(point-num=-1, sigma=4.0, heatmap_type=gaussian, length=0, dataset=300W-68)
Initialize cpm-vgg16 with configure : Configure(arch='cpm_vgg16', stages=3, dilation=[1], pooling=[True, True, True], downsample=8, argmax=4, pretrained=[True])
vgg16_base use pre-trained model
'detector'


In [12]:
sqlite3_path = "/home/equipo/Datasets/replayattack/replay_attack.sqlite3"
videos = ReplayAttackParser.get_db_records_as_frame(
                datasets=["train"], sqlite3_path=sqlite3_path
            )

In [22]:
videos

,path,label
0,Dataset/train/attack/hand/attack_mobile_client...,1
1,Dataset/train/attack/hand/attack_highdef_clien...,1
2,Dataset/train/attack/hand/attack_highdef_clien...,1
3,Dataset/train/attack/hand/attack_highdef_clien...,1
4,Dataset/train/attack/hand/attack_highdef_clien...,1
...,...,...
355,Dataset/train/real/client018_session01_webcam_...,0
356,Dataset/train/real/client016_session01_webcam_...,0
357,Dataset/train/real/client004_session01_webcam_...,0
358,Dataset/train/real/client006_session01_webcam_...,0


In [52]:
root_path = "/home/equipo/Datasets/replayattack/"
for index, video in videos.iterrows():
    path = os.path.join(root_path, video["path"])
    face_landmarks = {}
    if os.path.isfile(f'{path}.list') and not os.path.isfile(f'{path}.hdf5'):
        frames = get_frames(path)
        face_locs = pd.read_csv(
                f'{path}.list', sep=' ', header=None, index_col=0, names=['x', 'y', 'w', 'h'])
        prev_time = time.time()
        for index, frame in tqdm(frames.items()):
            try :
                [x, y, w, h] = face_locs.loc[index].to_numpy()
                pred = model.process_image(frame, [x, y, x + w, y + h])
                face_landmarks[index] = pred
            except:
                print(f'Index error {index}')
            
        current_time = time.time()
        inference_time = datetime.timedelta(
            seconds=current_time - prev_time)
        print("\t+ Inference Time: %s" % (inference_time))

        with h5py.File(f'{path}.hdf5', 'w') as hf:
            for index, landmark in tqdm(face_landmarks.items()):
                hf.create_dataset(str(index), data=landmark)


Processing /home/equipo/Datasets/replayattack/Dataset/train/attack/hand/attack_mobile_client016_session01_mobile_video_controlled.mov: 0.0 - 320 - 240


  0%|          | 0/231 [00:00<?, ?it/s]/home/equipo/mambaforge/envs/python_libs/lib/python3.7/site-packages/torch/nn/functional.py:4290: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  "Default grid_sample and affine_grid behavior has changed "
/home/equipo/mambaforge/envs/python_libs/lib/python3.7/site-packages/torch/nn/functional.py:4228: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  "Default grid_sample and affine_grid behavior has changed "
100%|█████████▉| 230/231 [01:18<00:00,  2.93it/s]


NameError: name 'e' is not defined

In [50]:
[x, y, w, h] = face_locs.loc[229].to_numpy()